In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from trading.operations import create_trading_app, setup_trading_app, place_trade, cancel_trade, accept_trade, close_trading
from store.operations import *
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

ALGOD_URL = os.environ.get('ALGOD_URL')
ALGOD_TOKEN = os.environ.get('ALGOD_TOKEN')
print(f"ALGOD_URL: {ALGOD_URL}")
print(f"ALGOD_TOKEN: {ALGOD_TOKEN}")

client = get_algod_client(ALGOD_URL, ALGOD_TOKEN)

ALGOD_URL: http://localhost:4001
ALGOD_TOKEN: aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))
print(f"storeAppID: {storeAppID}")
#close_trading(client, 693, storeAppID, creator)

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

storeAppID: 4734
Creator address: NBIIRJVCVD7ZWZGKCPLVZ5Z6QFUCXDE2GR4KQR5URM2ZBPV7V4VKPJFRZI
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI


Alice is generating an example token...

In [4]:
def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
    txn = transaction.AssetConfigTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        total=total,
        decimals=decimals,
        asset_name=asset_name,
        unit_name=unit_name,
        default_frozen=False,
        strict_empty_address_check=False,
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    response = wait_for_confirmation(client, signed_txn.get_txid())
    assert response.asset_index is not None and response.asset_index > 0
    return response.asset_index

asset_id = create_dummy_asset(seller, 100000000000000000, 0, "example token", "AVT")

# asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

Waiting for confirmation...
Transaction KQHV6USX74TO7MBIRGD4B7GQPKHJIJWIZENLKVCGMFFKLAI2SO5A confirmed in round 218492.
The token id is: 4964


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeAppID = int(os.environ.get("STORE_APP_ID"))

storeAppID = create_store_app(client, creator)
print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Waiting for confirmation...
Transaction SQGKFLKEOLQ6HLFGJQ3VDWGB4HV6VNTYJN5V6XDYYSJKAZTCIAWQ confirmed in round 218494.
Store App ID: 4965
Store App address: ZESS2GUDJCZG6UUDX464ZD2GSRLIVW7B2EULLVBWSCPBNCZ3UZHSYW3S24
Waiting for confirmation...
Transaction EOUIFVZ6QHIM5YZW2UN5YYCDRT223WZZU5UQQJUIYHSJ4MT6WRPA confirmed in round 218496.
Store App ID: 4965
Store App Address: ZESS2GUDJCZG6UUDX464ZD2GSRLIVW7B2EULLVBWSCPBNCZ3UZHSYW3S24


In [7]:

appID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction 4QWSB6XGW7NDILKLTF2CLSGBBD3DG2KHOAHHXUN4VNEXEHQSK7NQ confirmed in round 218498.
Waiting for confirmation...
Transaction MJXEYGFMUQXOMBMSCNU7WEMHHXS3QPKKHEN4DEAZ6YZ3AIHOG2EQ confirmed in round 218500.
App ID: 4967
App Address: G4HJHFMQHDPCKWEXOOY2DUB7ZY465GVEXVD3KE5CIU5N4IVTG4OIYRTU7A


In [8]:
set_up(client=client, creator=creator, app_id=storeAppID, trade_app_id=appID, bid_app_id=appID, auction_app_id=appID, distribution_app_id=appID)

Waiting for confirmation...
Transaction TRQCLNO5EFMBW6I4MEC5WKIXB5JQD6JZ7SHXN45CBRNU6GRIN3LQ confirmed in round 218502.


Alice is setting up and funding token auction...

In [9]:
setup_trading_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_id=asset_id,
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]
print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction 4WB3R33PW5GXZJGLBH5BX56ZAD2OVZKBMASNTJFNJZ2DQA43VSWA confirmed in round 218504.
Alice's algo balance:  10054595078745  algos


In [10]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 201000, 4964: 0}


In [11]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10051504089949  algos
Carla is placing bid for:  1000000  algos


In [12]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

print("Carla bid", trading_index)

store_app_id 4965
seller GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI opt in app 4965
Waiting for confirmation...
Transaction TH5ZYS3QE6WQBR5SJ2V36GGSMRO6CVCATWYA4KZ2NEAP3LSTTG6A confirmed in round 218506.
new address: DROJGIOI6TONGEGCR32FE3FLNNOYNYZYV7QQY2AVT6FG4DEM7BLRBKSCHA
new private_key: DIetWFQkL6eavUxeQf2D8k80NGcbqhV4Rq0Su36rFaQcXJMhyPTc0xDCjvRSbKtrXYbjOK/hDGgVn4puDIz4Vw==
new passphrase: thunder render clump earn furnace prefer under erosion cloth vital draft you hammer snack swamp present actor boring find raise word hello camera absent mixed
Waiting for confirmation...
Transaction O6KTKWM4NFEUDW4C2TSUBOET7SBEHJUI5SLKVEV5MDNR2W33D2XQ confirmed in round 218508.
Waiting for confirmation...
Transaction IIHADVSOSNUDM5GMFLK24CSKJNGGZRW7YUO5PVCNZFQXBP5JCFCA confirmed in round 218510.
Waiting for confirmation...
Transaction FRY2W2PFWIZBWS6GANNHBOFJMPEKNRW7ZQH7WV7VXEU24SUK4PAQ confirmed in round 218512.
token_txn: {'sender': 'GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7

In [13]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

cancel_trade(client, appID, seller, trading_index)
print("Carla cancel trading", trading_index)

Waiting for confirmation...
Transaction WQL2Y4KCGU45JGPXEJA7IJUU7YFDDKJV3UB5EM2TUA42LTUFM2EQ confirmed in round 218516.
Carla cancel trading DROJGIOI6TONGEGCR32FE3FLNNOYNYZYV7QQY2AVT6FG4DEM7BLRBKSCHA


In [14]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=15, price=bidPrice, trading_index=trading_index)

store_app_id 4965
new address: GJYKQBVDYARNMTKVHE2PXATHK3HI4S3ZGRIEUDPDG2QRDJCVBYXDRJWVNA
new private_key: f6x7/RyXpmy/Bpp6ABqJM+LwxQ6LHhqGofU4LGj3BVUycKgGo8Ai1k1VOTT7gmdWzo5LeTRQSg3jNqERpFUOLg==
new passphrase: morning upgrade leaf impact essay swamp curious old aunt gym setup case tide ramp flash burden select maid kiss seminar space lava popular able hurry
Waiting for confirmation...
Transaction 4T2MYCJJLL3CWV6MK5Q3PTAEDFYBWELQMZESCAYZBV4SPSSRKDEQ confirmed in round 218518.
Waiting for confirmation...
Transaction SWYXFLWITMVHYXCE5ANCMQNKR7IABUTY5JS46TU7YGYPAYSMPXAA confirmed in round 218520.
Waiting for confirmation...
Transaction 7TKA7BRIEQW2R7BCSHPC3N4TSRUZHPQWCI5DMAV2CKVSDNM32C6A confirmed in round 218522.
token_txn: {'sender': 'GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI', 'fee': 506000, 'first_valid_round': 218516, 'last_valid_round': 219516, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'S46yr60YjyJzK3Fk0J8V7mzMIOSxovaXAVqCHlwPl1A=', 'group': None, '

'GJYKQBVDYARNMTKVHE2PXATHK3HI4S3ZGRIEUDPDG2QRDJCVBYXDRJWVNA'

In [15]:
accept_trade(client, appID, bidder, seller.get_address(), trading_index)

token_amount 15
price 1000000
Waiting for confirmation...
Transaction AB4GUBNCPDXAOSCQISJF5YIMWLPOTO3LMLUXRZCRBABBM6VGM54A confirmed in round 218528.
Waiting for confirmation...
Transaction JOIVG5C2UYKYPMA3J56BZFYVTL7JWYA66BBGZWYPFC553VVVERFA confirmed in round 218530.
Waiting for confirmation...
Transaction D3GODDPROR67SWERHTYD254EXT6EXGEKKM6VMQHFPPSAOE3ZAYEQ confirmed in round 218532.


True

Alice is closing out the auction....

In [16]:
sleep(5)

close_trading(client, appID, creator, [asset_id])

b'assets' [4964]
b'accounts' ['NBIIRJVCVD7ZWZGKCPLVZ5Z6QFUCXDE2GR4KQR5URM2ZBPV7V4VKPJFRZI', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU']
Waiting for confirmation...
Transaction 4M6FXY7DKHCNJBMUYJZBH3R7XCGO3VDFRGCYM3Z2KPKZJHFRLUOQ confirmed in round 218535.


In [17]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 201000, 4964: 0}


In [19]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]
print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee

Carla's NFT balance: 15  for token ID:  4964
Alice's balances after auction:  {0: 10054613165931, 6: 0, 17: 0, 31: 0, 42: 0, 53: 0, 68: 0, 80: 0, 92: 0, 107: 0, 118: 0, 129: 0, 140: 0, 155: 0, 166: 0, 177: 0, 195: 1, 206: 0, 220: 1, 231: 1, 247: 1, 265: 1, 282: 1, 298: 1, 499: 999999999999700000, 4964: 99999999999999985}  Algos
Carla's balances after auction:  {0: 10051523183955, 499: 155, 2989: 50000000050050, 3559: 49999999900000, 3992: 0, 4002: 10, 4031: 10, 4050: 10, 4964: 15}  Algos


In [20]:
app_global_state = get_app_global_state(client, storeAppID)
print(app_global_state)
seller_app_local_state = get_app_local_state(client, storeAppID, seller.get_address())
print(seller_app_local_state)
bidder_app_local_state = get_app_local_state(client, storeAppID, bidder.get_address())
print(bidder_app_local_state)
# print(encoding.encode_address(app_global_state[b"AS"]))
# print(encoding.encode_address(app_global_state[b"S"]))
#print(encoding.encode_address(storeAppState[b"C"]))

{b'AA_ADDR': 4967, b'BA_ADDR': 4967, b'TA_ADDR': 4967, b'DA_ADDR': 4967, b'TBA': 1000000, b'TSA': 1000000}
{b'SA': 1000000}
{b'BA': 1000000}
